In [443]:
import os
import warnings
import requests
import string 
import numpy as np
import nltk
import time
from numpy import linalg as LA
import regex as re
import pandas as pd
from lda import LDA
from datetime import datetime
from bs4 import BeautifulSoup
from alive_progress import alive_bar
from gensim.parsing.preprocessing import remove_stopwords
from sklearn.feature_extraction.text import CountVectorizer
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from webdriver_manager.core.utils import ChromeType
from selenium.webdriver.chrome.options import Options
from alive_progress import alive_bar

warnings.simplefilter(action="ignore", category=FutureWarning)
warnings.simplefilter(action="ignore", category=RuntimeWarning)


In [444]:
ROOT_DIR = os.path.dirname(os.path.abspath("__file__"))
PARENT_DIR = os.path.dirname(ROOT_DIR)
GUARDIAN_DIR = os.path.join(ROOT_DIR, "data", "Guardian.csv")
REUTERS_DIR = os.path.join(ROOT_DIR, "data", "Reuters.csv")
CNN_DIR = os.path.join(PARENT_DIR, "data", "CNN.csv")
DAILYMAIL_DIR = os.path.join(PARENT_DIR, "data", "DailyMail.csv")
GLOVE_DIR = os.path.join(ROOT_DIR, "glove_data", "results", "vectors.txt")

# Daily Mail

In [447]:
class DailyMail: 
    def __init__(self) -> None:
        self.source = "DailyMail"

    def fromScratch(self):
            if not os.path.exists(DAILYMAIL_DIR):
                self.old_data = pd.DataFrame(columns=["Date", "URL", "Title", "Text"])
                self.from_scratch = True
            else:
                self.old_data = pd.read_csv(DAILYMAIL_DIR)
                self.from_scratch = False

    def concatData(self):
        result = pd.concat([self.old_data, self.new_data])
        result = result.drop_duplicates(subset=["Text"])
        result = result.set_index("Date")
        result = result.sort_index(ascending=False)
        return result

    def URLFetcher(self):
            self.urls = []
            self.dates = []

            if self.from_scratch == False:
                last_url = self.old_data.iloc[0,1]
            elif self.from_scratch == True:
                last_url = "https://www.dailymail.co.uk/news/article-10626865/Boris-plans-face-face-meeting-Biden-emergency-NATO-summit.html"

            with alive_bar(title=f"→ {self.source}: Fetching URLs in pages", bar=None, spinner="dots", force_tty=True) as bar:
                for page in range(0,5): # 95
                    leading_url = "https://www.dailymail.co.uk"
                    url = "https://www.dailymail.co.uk/home/search.html?offset="+str(page*50)+"&size=50&sel=site&searchPhrase=ukraine+russia&sort=recent&channel=news&type=article&days=all"
                    title_tag="sch-res-title"
                    exc_list=[]
                    inc_list=["/news/"]
                    try:
                        html_text = requests.get(url).text
                        soup = BeautifulSoup(html_text, "lxml")
                        headlines = soup.find_all("h3", class_=title_tag)
                        for headline in headlines:
                            _ = headline.find("a", href=True)
                            url = leading_url + _["href"]
                            self.urls.append(url)
                            if last_url == url:
                                break
                        if last_url == url:
                            break
                    except Exception as e:
                        print(f"Error in page {page}: {e}")
                        pass
                    bar()
            self.unique_urls = list(dict.fromkeys(self.urls))
            print(f"→ {len(self.unique_urls)} URLs fetched successfully!")

    def articleScraper(self):
        bodies = []
        titles = []
        dates = []
        urls = []
        rep = {"The Mail on Sunday can reveal:": "",
        "RELATED ARTICLES":"",
        "Share this article":""}

        def replaceAll(text, dic):
            for i, j in dic.items():
                text = text.replace(i, j)
            return text

        with alive_bar(len(self.unique_urls), title=f"→ {self.source}: Article scraper", spinner="dots_waves", bar="smooth", force_tty=True) as bar:
            for url in self.unique_urls:
                try:
                    title_tags = ["pg-headline"]
                    text_tags = ["mol-para-with-font"]
                    date_box_tag = ["article-timestamp article-timestamp-published"]
                    html_text = requests.get(url).text
                    soup = BeautifulSoup(html_text, "lxml")
                    title = soup.find("h2").text
                    date_box = soup.find("span", class_ = date_box_tag)
                    date = date_box.find("time")
                    paragraphs = soup.find_all("p", class_=text_tags)
                    body =""
                    for _ in paragraphs:
                        body += " " + _.text
                    body = replaceAll(body, rep)
                    bodies.append(" ".join(body.split()))
                    titles.append(title)
                    urls.append(url)
                    dates.append(date.get('datetime')[:10])
                    bar()
                except Exception as e:
                    print(f"URL couldn't be scraped: {url} because {e}")
                    pass
        data = pd.DataFrame({"URL": urls, "Date": dates, "Title": titles, "Text": bodies})
        self.new_data = data

    def scraper(self):
        self.fromScratch()
        self.URLFetcher()
        self.articleScraper()
        data = self.concatData()
        lenAfter = len(data) - len(self.old_data)
        if lenAfter == 0:
            print(f"→ No new articles found. Total articles: {len(data)}")
        else:
            print(f"→ {lenAfter} new articles saved to {self.source}.csv! Total articles: {len(data)}")
        data.to_csv(DAILYMAIL_DIR, index=True)

        return data

In [448]:
dailymail = DailyMail()
dailymail.scraper()

→ DailyMail: Fetching URLs in pages (!) 0 in 1.1s (0.00/s) 
→ 1 URLs fetched successfully!
→ DailyMail: Article scraper |████████████████████████████████████████| 1/1 [100%] in 1.8s (0.56/s)                     
→ No new articles found. Total articles: 102


,URL,Title,Text
Date,,,
2022-08-28,https://www.dailymail.co.uk/news/article-11154...,"'This guy's already dead, let him burn in hell...",This is the disgusting moment a pro-Putin war ...
2022-08-28,https://www.dailymail.co.uk/news/article-11153...,Anti-Putin pensioner who torched Russian gener...,A Russian anti-war protestor who set fire to a...
2022-08-28,https://www.dailymail.co.uk/news/article-11153...,Nearly three-quarters of state pension will be...,Pensioners face a 'terrifying' winter as risin...
2022-08-28,https://www.dailymail.co.uk/news/article-11153...,Deputy PM Dominic Raab played laser quest whil...,Deputy Prime Minister Dominic Raab enjoyed a g...
2022-08-28,https://www.dailymail.co.uk/news/article-11153...,Liz Truss 'mulls emergency tax cuts including ...,Lis Truss is reportedly mulling sweeping tax c...
...,...,...,...
2022-08-21,https://www.dailymail.co.uk/news/article-11130...,Russia proudly shows off an Australian Bushmas...,Russia has displayed a destroyed Australian-ma...
2022-08-21,https://www.dailymail.co.uk/news/article-11130...,Three Kremlin 'spies' are arrested after 'spra...,Three alleged Kremlin spies caught breaking in...
2022-08-20,https://www.dailymail.co.uk/news/article-11130...,Daughter of Ukraine war mastermind 'is blown t...,The daughter of Vladimir Putin's so-called 'Ra...
